# 2 ML Features Selection & Model Training



In [ ]:
# # Save Current Session
# import dill

# dill.dump_session('temp_save_session.pkl')

In [ ]:
# # Load a previous session
# import dill

# dill.load_session('temp_save_session.pkl')

# Imports & Functions

## Imports

In [ ]:
# General imports
import os
import sys
import gc
import warnings
from typing import Literal

from datetime import datetime
from pprint import pprint
import time
import pickle
import cloudpickle
import random
from collections import Counter

# Custom Functions
sys.path.append(os.path.abspath('../Notebooks/Utilities')) 
import cust_utilities as utils

# Maths, Pandas etc
import math
import numpy as np
import pandas as pd
import scipy as sci

# Plots
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.backends.backend_pdf import PdfPages

# ML Prep
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.compose import make_column_selector

# ML Training
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import matthews_corrcoef, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
from ml_utilities import ml_model_pipeline_details, grid_search_results, classification_metrics 
from ml_utilities import feature_importance, get_shap_importance, get_prediction_probabilities
import shap

# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier



## Results & Features Inspection

In [ ]:
# Plots for subject info
#

def subject_info_plot(subjects_df):

    # PD & Gender
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
    fig.suptitle('Study Subjects - PD & Gender', fontsize=18)

    counts = subjects_df['pd'].value_counts()
    axes[0].set_title('PD')
    axes[0].bar(counts.index.astype(str), counts.values, color=['skyblue', 'skyblue'], edgecolor='black')
    axes[0].set_xticks(range(len(counts.index)))
    axes[0].set_xticklabels(['Yes', 'No'])
    for i, (label, count) in enumerate(counts.items()):
        axes[0].text(i, count/2, str(count), ha='center', va='center', fontsize=12)

    counts = subjects_df['gender'].value_counts()
    axes[1].set_title('Gender')
    axes[1].bar(counts.index.astype(str), counts.values, color=['skyblue', 'skyblue'], edgecolor='black')
    axes[1].set_xticks(range(len(counts.index)))
    axes[1].set_xticklabels(['Male', 'Female'])
    for i, (label, count) in enumerate(counts.items()):
        axes[1].text(i, count/2, str(count), ha='center', va='center', fontsize=12)

    plt.tight_layout()
    plt.show()

    # Age Distribution & Box
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
    fig.suptitle('Study Subjects - Age Distribution', fontsize=18)

    axes[0].hist(subjects_df['age'], bins=15, color='skyblue', edgecolor='black')

    axes[1].set_xticks([0])
    box = axes[1].boxplot(subjects_df['age'].dropna(), patch_artist=True)
    for patch in box['boxes']:
        patch.set(facecolor='skyblue')

    plt.tight_layout()
    plt.show()


In [ ]:
# Plots for EEG Preprocessing Results
#

def eeg_preprocess_results_plot(results_df):

    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))
    fig.suptitle('EEG Preprocessing Metrics - All Subjects', fontsize=18)

    # Quality Warning
    counts = results_df['EEG_preprocessing_quality_warning'].value_counts().reindex([True, False], fill_value=0)
    axes[0].set_title('Overall Quality Warning Count')
    axes[0].bar(counts.index.astype(str), counts.values, color=['salmon', 'lightgreen'], edgecolor='black')
    axes[0].set_xticks(range(len(counts.index)))
    axes[0].set_xticklabels(['Yes', 'No'])
    for i, (label, count) in enumerate(counts.items()):
        axes[0].text(i, count/2, str(count), ha='center', va='center', fontsize=12)

    # # Channels count
    # counts = results_df['channel_count'].value_counts()
    # axes[1].set_title('Channels Count')
    # axes[1].bar(counts.index.astype(str), counts.values, color=['skyblue'], edgecolor='black')

    # ICA Rejection Level
    axes[1].set_title('ICA - ICs Rejection Level')
    box = axes[1].boxplot(results_df['ICA_rejection_level'].dropna(), patch_artist=True)
    for patch in box['boxes']:
        patch.set(facecolor='skyblue')

    # Epoch Rejection Level
    axes[2].set_title('Epoch Rejection Level')
    box = axes[2].boxplot(results_df['epoch_rejection_level'].dropna(), patch_artist=True)
    for patch in box['boxes']:
        patch.set(facecolor='skyblue')
    # axes[2].set_xlabel('Epoch Rejection Level')
    # axes[2].set_ylabel('Number of Subjects')
    # axes[2].hist(results_df['epoch_rejection_level'], bins=10, color='skyblue', edgecolor='black')
    # nonzero_epoch_rejection = results_df['epoch_rejection_level'][results_df['epoch_rejection_level'] > 0]
    # axes[2].hist(nonzero_epoch_rejection, bins=15, color='salmon', edgecolor='black', alpha=0.7)
    # for bar in axes[2].patches:
    #     bar.set_width(bar.get_width() * 0.8)

    plt.tight_layout()
    plt.show()


In [ ]:
# Plots for EEG SpecParam Results
#

def eeg_specparam_results_plot(results_df):

    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))
    fig.suptitle('EEG SpecParam Fit Metrics - All Subjects', fontsize=18)

    # Quality Warning
    counts = results_df['chn_SPM_fit_quality_warning'].value_counts().reindex([True, False], fill_value=0)
    axes[0].set_title('Overall Quality Warning Count')
    axes[0].bar(counts.index.astype(str), counts.values, color=['salmon', 'lightgreen'], edgecolor='black')
    
    axes[0].set_xticks(range(len(counts.index)))
    axes[0].set_xticklabels(['Yes', 'No'])
    for i, (label, count) in enumerate(counts.items()):
        axes[0].text(i, count/2, str(count), ha='center', va='center', fontsize=12)

    # Error Mean
    axes[1].set_title('Error Mean')
    box = axes[1].boxplot(results_df['chn_error_mean'].dropna(), patch_artist=True)
    for patch in box['boxes']:
        patch.set(facecolor='skyblue')

    # R-Squared Mean
    axes[2].set_title('R2 Mean')
    box = axes[2].boxplot(results_df['chn_r2_mean'].dropna(), patch_artist=True)
    for patch in box['boxes']:
        patch.set(facecolor='skyblue')

    plt.tight_layout()
    plt.show()

    # Number of flagged channels
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(3, 4))
    # fig.suptitle('EEG SpecParam Fit Metrics - Flagged Channels', fontsize=18)
    flagged_counts = results_df['chn_flagged_channels'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    axes.set_title('Flagged Channels Count')
    box = axes.boxplot(flagged_counts, patch_artist=True)
    for patch in box['boxes']:
        patch.set(facecolor='salmon')

    plt.tight_layout()
    plt.show()

In [ ]:
# Plots for Aperiodic Features Spread
#

def aperiodic_features_plot(results_df, level):

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
    fig.suptitle(f'Aperiodic Features - {level}', fontsize=18)

    # Offset
    axes[0].set_title('Offset')
    box = axes[0].boxplot(results_df['offset'].dropna(), patch_artist=True)
    for patch in box['boxes']:
        patch.set(facecolor='skyblue')

    # Exponent
    axes[1].set_title('Exponent')
    box = axes[1].boxplot(results_df['exponent'].dropna(), patch_artist=True)
    for patch in box['boxes']:
        patch.set(facecolor='skyblue')

    plt.tight_layout()
    plt.show()


In [ ]:
# Plots for Periodic Features Spread
#

def periodic_features_plot(results_df, level):

    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))
    fig.suptitle(f'Periodic Features - {level}', fontsize=18)

    # CF
    axes[0].set_title('CF_0')
    box = axes[0].boxplot(results_df['cf_0'].dropna(), patch_artist=True)
    for patch in box['boxes']:
        patch.set(facecolor='skyblue')

    # PW
    axes[1].set_title('PW_0')
    box = axes[1].boxplot(results_df['pw_0'].dropna(), patch_artist=True)
    for patch in box['boxes']:
        patch.set(facecolor='skyblue')

    # BW
    axes[2].set_title('BW_0')
    box = axes[2].boxplot(results_df['bw_0'].dropna(), patch_artist=True)
    for patch in box['boxes']:
        patch.set(facecolor='skyblue')

    plt.tight_layout()
    plt.show()


In [ ]:
# Plots for Aperiodic Peaks Count 
#

def periodic_peaks_plot(results_df):

    regions_df = results_df[results_df['channel'].isnull()].copy()
    cf_cols = [col for col in regions_df.columns if col.startswith('cf_')]
    regions_df['num_periodic_cf'] = regions_df[cf_cols].notnull().sum(axis=1)
    
    channels_df = results_df[results_df['region'].isnull()].copy()
    cf_cols = [col for col in channels_df.columns if col.startswith('cf_')]
    channels_df['num_periodic_cf'] = channels_df[cf_cols].notnull().sum(axis=1)

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
    fig.suptitle(f'Periodic Peaks Count', fontsize=18)

    # Region
    axes[0].set_title('Regions')
    box = axes[0].boxplot(regions_df['num_periodic_cf'].dropna(), patch_artist=True)
    for patch in box['boxes']:
        patch.set(facecolor='skyblue')

    # Channel
    axes[1].set_title('Channels')
    box = axes[1].boxplot(channels_df['num_periodic_cf'].dropna(), patch_artist=True)
    for patch in box['boxes']:
        patch.set(facecolor='skyblue')

    plt.tight_layout()
    plt.show()


# Run: 1. Study Load & Inspections

In [ ]:
# Study and Processing Run Details

#---- Parameters --------------------------------
# Study & Processing Run Details
study_name = 'IOWA_Rest'
eeg_features_run = '1b_EEG_Features_Results_Run_20250801_full_run'

run_description = 'test_regions_aperiodic'
# test_mode = False

# Extraction Parameters
extraction_params = {'features_detail_level': 'region',    # region or channel, default channels
                     'subject_meta_include': False,
                     'aperiodic_include': True,
                     'periodic_include': False
                    }
model_training_params = {'EEG_features_source_run': eeg_features_run}
#----------------------------------------------------

# Get existing study details, if exists
study_folder_path = utils.get_folder_path('Study_' + study_name)
study_info = pd.read_pickle(study_folder_path + '/study_inf.pkl', compression='zip')
study_subjects_df = pd.read_pickle(study_folder_path + '/study_subjects_df.pkl', compression='zip')

# EEG Processing Results Data
eeg_features_run_results_path = utils.get_folder_path(study_info['eeg_processing_results_path'] + '/' + eeg_features_run)
eeg_features_run_details = pd.read_pickle(eeg_features_run_results_path + '/run_details.pkl', compression='zip')
eeg_preprocessing_run = eeg_features_run_details['eeg_preprocessed_data']

eeg_preprocessing_run_results_path = utils.get_folder_path(study_info['eeg_processing_results_path'] + '/' + eeg_preprocessing_run)
eeg_preprocessed_data_path = utils.get_folder_path(eeg_preprocessing_run_results_path + '/Cleaned_files' )
eeg_preprocessing_run_details = pd.read_pickle(eeg_preprocessing_run_results_path + '/run_details.pkl', compression='zip')
eeg_processing_results_df = pd.read_pickle(eeg_features_run_results_path + '/eeg_processing_results_df.pkl', compression='zip')
eeg_features_superset_df = pd.read_pickle(eeg_features_run_results_path + '/eeg_features_superset_df.pkl', compression='zip')
eeg_features_flattened_df = pd.read_pickle(eeg_features_run_results_path + '/eeg_features_flattened_df.pkl', compression='zip')

# Setup the extraction run and results folder & save params
current_date = datetime.now().strftime('%Y%m%d')
run_name = f'2_Feature_Selection_Training_Run_{current_date}_{run_description}'
run_results_path = utils.extend_folder_path(study_info['ml_training_results_path'], run_name, exists_ok=False)

run_details = pd.Series({
    'study_name': study_name,
    'run_name': run_name,
    'extraction_params': extraction_params,
    'model_training_params': model_training_params
    })
run_details.to_pickle(run_results_path + '/run_details.pkl', compression='zip')

# # Set progress messages, testing
# if test_mode:
#     VERBOSE = True
#     TEST_SUBJECTS = [0,5,101]
#     # TEST_CHANNELS = ['F5', 'C3', 'P3', 'F6', 'C6', 'P6']
# else:
#     VERBOSE = False
#     TEST_SUBJECTS = []
#     # TEST_CHANNELS = []

In [ ]:
# Processing Run Details & Data Structures
summary = f'EEG Processing Parameters'
summary = summary + f"\n- Study: {study_info['study_name']} {study_info['dataset_ref']}"
summary = summary + f"\n- EEG Processing Run: {eeg_preprocessing_run_details['run_name']}"
summary = summary + f"\n-   Preprocess Params: {eeg_preprocessing_run_details['preprocess_params']}"
summary = summary + f"\n-   ICA Params: {eeg_preprocessing_run_details['artefact_params']}"
summary = summary + f"\n- EEG Features Run: {eeg_features_run}"
summary = summary + f"\n-   PSD Params: {eeg_features_run_details['psd_params']}"
summary = summary + f"\n-   SpecParam Params: {eeg_features_run_details['specparam_params']}"
summary = summary + f"\n- Features Selection Run: {run_name}"
summary = summary + f"\n-   Feature Selection Params: {run_details['extraction_params']}"
summary = summary + f"\n-   Model Training Params: {run_details['model_training_params']}"

print(f'{summary}\n')

# Processing Metrics
print('EEG Processing Results')
print(eeg_processing_results_df.shape)
display(eeg_processing_results_df.head())

print(f'Null Fits: {sum(eeg_processing_results_df["chn_null_fits"])}')
eeg_preprocess_results_plot(eeg_processing_results_df)
eeg_specparam_results_plot(eeg_processing_results_df)


In [ ]:
# Study Subjects Summary
print('Study Subjects')
print(study_subjects_df.shape)
display(study_subjects_df.head())

subject_info_plot(study_subjects_df)

In [ ]:
# %pip install selenium
# %pip install great_tables[extra]

# from great_tables import GT
# GT(study_subjects_df.head(10)).tab_header(title="Sample Table (First 10 Rows)").fmt_number(columns="value_column").save('temp.png')

In [ ]:
# Features Superset 
print('EEG Features Superset')
print(eeg_features_superset_df.shape)
display(eeg_features_superset_df.head())

# Features Flattened 
print('EEG Features Flattened')
print(eeg_features_flattened_df.shape)
display(eeg_features_flattened_df.head())


In [ ]:
# Features - At Region & Channel Detail Level
#

regions_df = eeg_features_superset_df[eeg_features_superset_df['channel'].isnull()].copy()
channels_df = eeg_features_superset_df[eeg_features_superset_df['region'].isnull()].copy()

aperiodic_features_plot(regions_df, 'Regions')
aperiodic_features_plot(channels_df, 'Channels')

periodic_features_plot(regions_df, 'Regions')
periodic_features_plot(channels_df, 'Channels')

periodic_peaks_plot(eeg_features_superset_df)

# Run: 2. Split X & Y

In [ ]:
# Separate X features and y target
# For Data Pipeline & ML Model Training
#

target_col_name = 'pd'
feature_names = eeg_features_flattened_df.columns[eeg_features_flattened_df.columns != target_col_name]
X = eeg_features_flattened_df[feature_names].copy()
y = eeg_features_flattened_df[target_col_name].copy()

# Data Split : Training & Test, 80:20. NB cross-validation will be performed using Training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TODO: Different split?
# split by *subjects*, not by raw rows
# train_subj, test_subj = train_test_split(subjects, stratify=labels,
#                                          test_size=.3, random_state=42)
# X_train = eeg_long[eeg_long.subject_id.isin(train_subj)]
# X_test  = eeg_long[eeg_long.subject_id.isin(test_subj)]
# y_train = labels.loc[train_subj].values
# y_test  = labels.loc[test_subj].values

# Run: 3. Feature Selection, Data Cleaning & Pipeline Save

In [ ]:
# Feature Selection using a Custom Transformer class
#

class FeatureSelection(BaseEstimator, TransformerMixin):

    def __init__(self, features_detail_level='default', selected_features='default'):
        # Parameters for the selection
        self.features_detail_level = features_detail_level
        self.selected_features = selected_features

    def fit(self, X, y=None):
        if isinstance(X, pd.DataFrame):
            X_temp_df = X.copy()
        else:
            raise ValueError("X must be a pandas DataFrame for feature selection.")
        
        # Filter out features according to detail level, ie region or channel
        drop_cols =[]
        if self.features_detail_level == 'region':
            drop_cols = drop_cols + [col for col in X_temp_df.columns if col.startswith('channel')]
        elif self.features_detail_level == 'channel':
            drop_cols = drop_cols +[col for col in X_temp_df.columns if col.startswith('region')]
        else:
            raise ValueError(f'Detail of {self.features_detail_level} is not region or channel')
        X_temp_df = X_temp_df.drop(columns=drop_cols, errors='ignore')
        
        # Only retain columns whose names contain any of the selected features
        X_temp_df = X_temp_df[[col for col in X_temp_df.columns if any(feat in col for feat in self.selected_features)]]
       
        self.selected_features_ = list(X_temp_df)
        return self
    
    def transform(self, X, y=None):
        if isinstance(X, pd.DataFrame):
            X_temp_df = X.copy()
        else:
            raise ValueError("X must be a pandas DataFrame for feature selection.")
        
        # Apply feature selection
        return X[self.selected_features_]


In [ ]:
# Features Selection & Cleaning Pipeline Establish & Train & Save
#

# TODO: Compare no scaling at all and model performance
# TODO: Different scaling approaches?

# Transformer for Features Selection
detail_level = extraction_params['features_detail_level']
selected_features = []
if extraction_params['subject_meta_include']: selected_features += ['age', 'gender']
if extraction_params['aperiodic_include']: selected_features += ['offset', 'exponent']
if extraction_params['periodic_include']: selected_features += ['pw', 'bw', 'cf']

feature_selector =  FeatureSelection(detail_level, selected_features)

# Sub pipeline for numerical and categorical transformations
num_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='constant', fill_value=0)),
    # ('scale_num', RobustScaler())
    ('scale_num', RobustScaler(quantile_range=(10.0, 90.0)))
    ])
cat_pipeline = Pipeline([
    # ("impute", SimpleImputer(strategy="most_frequent")),
    # ("encode_cat", OneHotEncoder(handle_unknown="ignore", sparse=False))
    ('encode_cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist'))
    ])
cols_transform = ColumnTransformer([
    ('numeric', num_pipeline, make_column_selector(dtype_include=['float64', 'int64'])),
    ('categorical', cat_pipeline, make_column_selector(dtype_include='object'))
])

# Setup & train/fit the overall pipeline
features_prep_pipeline = Pipeline([
    ('features_selection', feature_selector),
    # ('drop_columns', dropper),
    ('data_preprocess', cols_transform)
    ])  
features_prep_pipeline.fit(X_train)

# Save the Data Prep Pipeline
# Using cloudpickle in order to handle the custom transformer class
cloudpickle.dump(features_prep_pipeline, open(run_results_path + '/' + 'features_prep_pipeline.pkl', 'wb'))


In [ ]:
# # Checkpoint - Pipeline

# print("Feature Extraction Pipeline Steps:")
# for name, step in features_prep_pipeline.named_steps.items():
#     print(f"- {name}: {step}")

# print("\nColumnTransformer Details:")
# ct = features_prep_pipeline.named_steps['data_preprocess']
# for name, trans, cols in ct.transformers_:
#     print(f"- Transformer: {name}")
#     print(f"    Columns: {cols}")
#     print(f"    Transformer object: {trans}\n")

# print("\nAll Pipeline Parameters")
# for param, value in features_prep_pipeline.get_params().items():
#     print(f"- {param}: {value}")

# Run: 4. Models Training on Processed Data

## Data Transform With Pipeline

In [ ]:
# Apply the transformations to X data & Inspect Results
#

X_train_transformed = features_prep_pipeline.transform(X_train)
X_test_transformed = features_prep_pipeline.transform(X_test)

# TODO: Examine the impact of scaling etc more closely eg some boxplots before and after .... how well has eg scaling worked?

# Before & After Datasets
print("Features Extraction vs Transformed Data Shapes")
print(f'- Original Features Selection: {eeg_features_flattened_df.shape}')
print(f'- Original X_train: {X_train.shape} and y_train: {y_train.shape}')
print(f'- Original X_test: {X_test.shape} and y_test: {y_test.shape}')
print(f'- Transformed X_train: {X_train_transformed.shape}')
print(f'- Transformed X_test: {X_test_transformed.shape}')

# Data Pipeline - Features Transformed
print("\nTransformed Features:")
for name, transformer, columns in cols_transform.transformers_:
    print(f"- {name}: {len(columns) if hasattr(columns, '__len__') else 'Unknown'} columns")

feature_names = features_prep_pipeline.named_steps['data_preprocess'].get_feature_names_out()
print(f'Feature Names: {len(feature_names)}')
# print(feature_names)
temp_names_X_train_df = pd.DataFrame(X_train_transformed, columns=feature_names)
temp_names_X_train_df.reset_index(drop=True, inplace=True)
print(temp_names_X_train_df.shape)
display(temp_names_X_train_df.head())


## Random Forest

In [ ]:
# Establish a Model Pipeline - Using Processed Data used in Pipeline Fitting
#

# Pipeline, params & grid search define
model_pipeline = Pipeline([
    ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1, verbose=False))
    ])

grid_params = {
    'classifier__criterion': ['gini', 'entropy', 'log_loss'],     # Default gini. Tree split evaluation function
    'classifier__n_estimators': [100, 150],                       # Default 100. Number of trees
    'classifier__max_depth': [3, 10, 15],                         # Default none, unlimited ... but don't want to overfit
    # 'classifier__max_features': [0.8, 2],           # Default is sqrt
    'classifier__max_leaf_nodes': [50, None],                     # Default none, unlimited
    # 'randomforestclassifier__min_samples_split': [2, 5],
    # 'classifier__class_weight': [None, 'balanced']              # Balanced gives more importance to minority classes ... ?? Improves recall at the expense of precision
    } 

# # Straight k-fold cross-validation
# grid_search = GridSearchCV(
#     model_pipeline, grid_params, cv=5, n_jobs=-1,
#     scoring='precision'
#     )

# Stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    model_pipeline, grid_params, cv=skf, n_jobs=-1,
    scoring='precision'
    # scoring='f1'
    # scoring='roc_auc'
    )


# Grid search run
start_time = time.perf_counter()
grid_search.fit(X_train_transformed, y_train)
duration = time.perf_counter() - start_time

# Checkpoint - Model Training Details
ml_model_pipeline_details(model_pipeline)
print('\n')
grid_search_results(grid_search, duration)
print(f'Best Model: \n{grid_search.best_estimator_}')

# Retain the best model & save it, as well aas the grid search
model_randforest = grid_search.best_estimator_

cloudpickle.dump(grid_search, open(run_results_path + '/' + 'grid_search_randforest.pkl', 'wb'))
cloudpickle.dump(model_randforest, open(run_results_path + '/' + 'model_randforest.pkl', 'wb'))


In [ ]:
# Model Predictions & Evaluation
#

start_time = time.perf_counter()
y_pred = model_randforest.predict(X_test_transformed)
y_probablities = model_randforest.predict_proba(X_test_transformed)
duration = time.perf_counter() - start_time

# Evaluate Prediction Results
print('Model Prediction Results')
print(f'Run Time: {duration:.4f}')
classification_metrics(model_randforest, X_test_transformed, y_test, y_pred)

# Assign Probabilities to Each Prediction
get_prediction_probabilities(y_pred, y_probablities, y_test)

# Feature Importance / Contribution
feature_importance(model_randforest, features_prep_pipeline)
features_importance_shap_df = get_shap_importance(X_train_transformed, X_test_transformed, model_randforest, features_prep_pipeline)



In [ ]:
# def make_predictions_custom_threshold(model, X, threshold=0.5):
#     """
#     Make predictions with custom threshold to minimize false positives
#     Higher threshold = fewer false positives, but potentially lower recall
#     """
#     probabilities = model.predict_proba(X)
#     predictions = (probabilities[:, 1] >= threshold).astype(int)
#     return predictions, probabilities


# predictions, probabilities = make_predictions_custom_threshold(
#     model_randforest, X_test_transformed, threshold=0.65
# )

# print(predictions)
# print(probabilities[:10])

# classification_metrics(model_randforest, X_test_transformed, y_test, predictions)

# print("Best parameters:", grid_search.best_params_)
# print("Best cross-validation score:", grid_search.best_score_)

In [ ]:
# from sklearn.metrics import precision_score, recall_score

# def find_optimal_threshold(model, X, y_true, metric='precision'):
#     """Find threshold that optimizes a specific metric"""
#     probabilities = model.predict_proba(X)[:, 1]
#     thresholds = np.arange(0.1, 1.0, 0.05)
    
#     best_threshold = 0.5
#     best_score = 0
    
#     for threshold in thresholds:
#         preds = (probabilities >= threshold).astype(int)
        
#         if metric == 'precision':
#             score = precision_score(y_true, preds, zero_division=0)
#         elif metric == 'recall':
#             score = recall_score(y_true, preds, zero_division=0)
        
#         if score > best_score:
#             best_score = score
#             best_threshold = threshold
    
#     return best_threshold, best_score

# best_theshold, best_score = find_optimal_threshold(model_randforest, X_test_transformed,y_test)
# print(best_theshold, best_score)

## Logistic Regression

In [ ]:
# Logistic Regression

# Pipeline, params & grid search define
model_pipeline = Pipeline([
    ('classifier', LogisticRegression(random_state=42))
    ])

grid_params = {
    'classifier__solver': ['saga','liblinear', 'lbfgs'],          # Default is lbfgs. Saga with L1 or l2. Large dataset. liblinear small datasets, binary classifications
    'classifier__penalty': ['l1', 'l2'],                        # Types of regulaisation
    'classifier__C': [1, 10, 100],                 # Default 1. Strength of regularisation, smaller is stronger
    'classifier__class_weight': [None, 'balanced'],           # Default None. Balanced seems to prevent near zero True predictions
    'classifier__max_iter': [1000, 2500, 5000]                      # Default 100. 
}
# scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

# # Straight k-fold cross-validation
# grid_search = GridSearchCV(
#     model_pipeline, grid_params, cv=5, n_jobs=-1,
#     scoring='precision'
#     )

# Stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    model_pipeline, grid_params, cv=skf, n_jobs=-1,
    scoring='precision'
    )

# Grid search run
start_time = time.perf_counter()
grid_search.fit(X_train_transformed, y_train)
duration = time.perf_counter() - start_time

# Checkpoint - Model Training Details
ml_model_pipeline_details(model_pipeline)
print('\n')
grid_search_results(grid_search, duration)
print(f'Best Model: \n{grid_search.best_estimator_}')

# Retain the best model & save it, as well aas the grid search
model_logreg = grid_search.best_estimator_

cloudpickle.dump(grid_search, open(run_results_path + '/' + 'grid_search_logreg.pkl', 'wb'))
cloudpickle.dump(model_logreg, open(run_results_path + '/' + 'model_logreg.pkl', 'wb'))


In [ ]:
# Model Predictions & Evaluation
#

start_time = time.perf_counter()
y_pred = model_logreg.predict(X_test_transformed)
duration = time.perf_counter() - start_time

# Evaluate Prediction Results
print('Model Prediction Results')
print(f'Run Time: {duration:.4f}')
classification_metrics(model_logreg, X_test_transformed, y_test, y_pred)

# Assign Probabilities to Each Prediction
get_prediction_probabilities(y_pred, y_probablities, y_test)

# Feature Importance / Contribution
features_importance_shap_df = get_shap_importance(X_train_transformed, X_test_transformed, model_logreg, features_prep_pipeline)


In [ ]:
# # Model Predictions & Evaluation - With Thresholds
# #

# start_time = time.perf_counter()
# y_pred = model_logreg.predict(X_test_transformed)
# y_pred_prob = model_logreg.predict_proba(X_test_transformed)[:,1]
# pred_threshold = 0.6
# y_pred_threshold = (y_pred_prob > pred_threshold).astype(int)
# duration = time.perf_counter() - start_time

# # for i in range(25):
# #     print(f' Binary: {y_pred[i]} vs Prob: {y_pred_prob[i]:.2f} vs {y_pred_threshold[i]}')

# # Evaluate Prediction Results
# print('Model Prediction Results')
# print(f'Run Time: {duration:.4f}')
# classification_metrics(model_logreg, X_test_transformed, y_test, y_pred_threshold)

## MLP Classifier

In [ ]:
# MLP Classifier

# Pipeline, params & grid search define
model_pipeline = Pipeline([
    ('classifier', MLPClassifier(random_state=42))
    ])

grid_params = {
    'classifier__hidden_layer_sizes': [(50,50), (100,50)],             # Default, single layer of 100
    # 'classifier__hidden_layer_sizes': [(50,), (100,), (50,50)],             # Default, single layer of 100
    'classifier__activation': ['tanh', 'relu'],                      # Default: relu
    # 'classifier__solver': ['adam', 'lbfgs', 'sgd'],                  # Default adam
    'classifier__solver': ['adam'],                  # Default adam
    'classifier__learning_rate_init': [0.00001, 0.0001, 0.001],
    'classifier__alpha': [0.1],                              # default L2, 0.0001,
    # 'classifier__early_stopping': [True],
    'classifier__batch_size': ['auto'],
    'classifier__max_iter': [1000]                            # Default 200
}

# # Straight k-fold cross-validation
# grid_search = GridSearchCV(
#     model_pipeline, grid_params, cv=5, n_jobs=-1,
#     scoring='precision'
#     )

# Stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    model_pipeline, grid_params, cv=skf, n_jobs=-1,
    scoring='precision'
    )

# Grid search run
start_time = time.perf_counter()
grid_search.fit(X_train_transformed, y_train)
duration = time.perf_counter() - start_time

# Checkpoint - Model Training Details
ml_model_pipeline_details(model_pipeline)
print('\n')
grid_search_results(grid_search, duration)
print(f'Best Model: \n{grid_search.best_estimator_}')

# Retain the best model & save it, as well aas the grid search
model_mlpc = grid_search.best_estimator_

cloudpickle.dump(grid_search, open(run_results_path + '/' + 'grid_search_mlpc.pkl', 'wb'))
cloudpickle.dump(model_mlpc, open(run_results_path + '/' + 'model_mlpc.pkl', 'wb'))

In [ ]:
# Model Predictions & Evaluation
#

start_time = time.perf_counter()
y_pred = model_mlpc.predict(X_test_transformed)
duration = time.perf_counter() - start_time

# Evaluate Prediction Results
print('Model Prediction Results')
print(f'Run Time: {duration:.4f}')
classification_metrics(model_mlpc, X_test_transformed, y_test, y_pred)

# Assign Probabilities to Each Prediction
get_prediction_probabilities(y_pred, y_probablities, y_test)

# Feature Importance / Contribution
features_importance_shap_df = get_shap_importance(X_train_transformed, X_test_transformed, model_mlpc, features_prep_pipeline)


In [ ]:
# # Model Predictions & Evaluation - With Thresholds
# #

# start_time = time.perf_counter()
# y_pred = model_mlpc.predict(X_test_transformed)
# y_pred_prob = model_mlpc.predict_proba(X_test_transformed)[:,1]
# pred_threshold = 0.6
# y_pred_threshold = (y_pred_prob > pred_threshold).astype(int)
# duration = time.perf_counter() - start_time

# # for i in range(25):
# #     print(f' Binary: {y_pred[i]} vs Prob: {y_pred_prob[i]:.2f} vs {y_pred_threshold[i]}')

# # Evaluate Prediction Results
# print('Model Prediction Results')
# print(f'Run Time: {duration:.4f}')
# classification_metrics(model_mlpc, X_test_transformed, y_test, y_pred_threshold)

# Run: 5. Models Training Separate or Combined Grid Search

## Temp Functions

In [ ]:
# Random Forest Model Training Function
#

def model_train_RandForest(X_train, y_train, feature_selector=None, cols_transform=None):

    # Use the data pipeline in the search
    if feature_selector:
        model_pipeline = Pipeline([
            ('features_selection', feature_selector),
            ('data_preprocess', cols_transform),
            ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1, verbose=False))
            ])

        grid_params = {
            'features_selection__features_detail_level': ['region'],
            'features_selection__selected_features': [['exp', 'offset']],
            'classifier__criterion': ['gini', 'entropy', 'log_loss'],     # Default gini. Tree split evaluation function
            'classifier__n_estimators': [100, 150],                       # Default 100. Number of trees
            'classifier__max_depth': [3, 10, 15],                         # Default none, unlimited ... but don't want to overfit
            # 'classifier__max_features': [0.8, 2],                       # Default is sqrt
            'classifier__max_leaf_nodes': [50, None],                     # Default none, unlimited
            # 'randomforestclassifier__min_samples_split': [2, 5],
            # 'classifier__class_weight': [None, 'balanced']              # Balanced gives more importance to minority classes ... ?? Improves recall at the expense of precision
            }
    else:
        model_pipeline = Pipeline([
            ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1, verbose=False))
            ])

        grid_params = {
            'classifier__criterion': ['gini', 'entropy', 'log_loss'],     # Default gini. Tree split evaluation function
            'classifier__n_estimators': [100, 150],                       # Default 100. Number of trees
            'classifier__max_depth': [3, 10, 15],                         # Default none, unlimited ... but don't want to overfit
            # 'classifier__max_features': [0.8, 2],                       # Default is sqrt
            'classifier__max_leaf_nodes': [50, None],                     # Default none, unlimited
            # 'randomforestclassifier__min_samples_split': [2, 5],
            # 'classifier__class_weight': [None, 'balanced']              # Balanced gives more importance to minority classes ... ?? Improves recall at the expense of precision
            }
    
    # # Straight k-fold cross-validation
    # grid_search = GridSearchCV(
    #     model_pipeline, grid_params, cv=5, n_jobs=-1,
    #     scoring='precision'
    #     )

    # Stratified k-fold cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    grid_search = GridSearchCV(
        model_pipeline, grid_params, cv=skf, n_jobs=-1,
        scoring='precision'
        # scoring='f1'
        # scoring='roc_auc'
        )

    # Grid search run
    start_time = time.perf_counter()
    grid_search.fit(X_train, y_train)
    duration = time.perf_counter() - start_time

    # Trace - Model Training Details
    ml_model_pipeline_details(model_pipeline)
    print('')
    grid_search_results(grid_search, duration)
    print(f'Best Model: \n{grid_search.best_estimator_}')

    # Retain the best model & save it, as well as the grid search
    model_randforest = grid_search.best_estimator_
    cloudpickle.dump(grid_search, open(run_results_path + '/' + 'grid_search_randforest.pkl', 'wb'))
    cloudpickle.dump(model_randforest, open(run_results_path + '/' + 'model_randforest.pkl', 'wb'))

    return model_randforest


In [ ]:
# Model Evaluation
#

def eval_model(model, data_pipeline, X_train, X_test, y_test):

    start_time = time.perf_counter()
    y_pred = model.predict(X_test)
    y_probablities = model.predict_proba(X_test)
    duration = time.perf_counter() - start_time

    # Evaluate Prediction Results
    print('Model Prediction Results')
    print(f'Run Time: {duration:.4f}')
    classification_metrics(model, X_test, y_test, y_pred)

    # Assign Probabilities to Each Prediction
    get_prediction_probabilities(y_pred, y_probablities, y_test)

    # Feature Importance / Contribution
    # feature_importance(model, data_pipeline)
    features_importance_shap_df = get_shap_importance(X_train, X_test, model, data_pipeline)


## Data Transform & Models With Separate Pipeline

In [ ]:
# Apply the transformations to X data & Inspect Results
#

X_train_transformed = features_prep_pipeline.transform(X_train)
X_test_transformed = features_prep_pipeline.transform(X_test)

# TODO: Examine the impact of scaling etc more closely eg some boxplots before and after .... how well has eg scaling worked?

# Before & After Datasets
print("Features Extraction vs Transformed Data Shapes")
print(f'- Original Features Selection: {eeg_features_flattened_df.shape}')
print(f'- Original X_train: {X_train.shape} and y_train: {y_train.shape}')
print(f'- Original X_test: {X_test.shape} and y_test: {y_test.shape}')
print(f'- Transformed X_train: {X_train_transformed.shape}')
print(f'- Transformed X_test: {X_test_transformed.shape}')

# Data Pipeline - Features Transformed
print("\nTransformed Features:")
for name, transformer, columns in cols_transform.transformers_:
    print(f"- {name}: {len(columns) if hasattr(columns, '__len__') else 'Unknown'} columns")

feature_names = features_prep_pipeline.named_steps['data_preprocess'].get_feature_names_out()
print(f'Feature Names: {len(feature_names)}')
# print(feature_names)
temp_names_X_train_df = pd.DataFrame(X_train_transformed, columns=feature_names)
temp_names_X_train_df.reset_index(drop=True, inplace=True)
print(temp_names_X_train_df.shape)
display(temp_names_X_train_df.head())


In [ ]:
model_randforest = model_train_RandForest(X_train_transformed, y_train)
eval_model(model_randforest, features_prep_pipeline, X_train_transformed, X_test_transformed, y_test)

## Combined Pipeline & Models Train

# xxxxxxx

In [43]:
# Transform ALL data, before feature selection
#

# Sub pipeline for numerical and categorical transformations
num_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='constant', fill_value=0)),
    # ('scale_num', RobustScaler())
    ('scale_num', RobustScaler(quantile_range=(10.0, 90.0)))
    ])
cat_pipeline = Pipeline([
    # ("impute", SimpleImputer(strategy="most_frequent")),
    # ("encode_cat", OneHotEncoder(handle_unknown="ignore", sparse=False))
    ('encode_cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist'))
    ])
cols_transform = ColumnTransformer([
    ('numeric', num_pipeline, make_column_selector(dtype_include=['float64', 'int64'])),
    ('categorical', cat_pipeline, make_column_selector(dtype_include='object'))
])

# Setup & train/fit the pipeline
X_train_temp = X_train.drop('subject_id', axis=1).copy()
X_test_temp = X_test.drop('subject_id', axis=1).copy()
data_prep_pipeline = Pipeline([
    ('data_preprocess', cols_transform)
    ])  
data_prep_pipeline.fit(X_train_temp)

# Apply to X Data
X_train_transformed = data_prep_pipeline.transform(X_train_temp)
X_test_transformed = data_prep_pipeline.transform(X_test_temp)

# Before & After Datasets
print("Features Extraction vs Transformed Data Shapes")
print(f'- Original Features Selection: {eeg_features_flattened_df.shape}')
print(f'- Original X_train: {X_train.shape} and y_train: {y_train.shape}')
print(f'- Original X_test: {X_test.shape} and y_test: {y_test.shape}')
print(f'- Transformed X_train: {X_train_transformed.shape}')
print(f'- Transformed X_test: {X_test_transformed.shape}')

# Data Pipeline - Features Transformed
print("\nTransformed Features:")
for name, transformer, columns in cols_transform.transformers_:
    print(f"- {name}: {len(columns) if hasattr(columns, '__len__') else 'Unknown'} columns")

feature_names = data_prep_pipeline.named_steps['data_preprocess'].get_feature_names_out()
print(f'Feature Names: {len(feature_names)}')
# print(feature_names)
temp_names_X_train_df = pd.DataFrame(X_train_transformed, columns=feature_names)
temp_names_X_train_df.reset_index(drop=True, inplace=True)
print(temp_names_X_train_df.shape)
display(temp_names_X_train_df.head())

In [ ]:
# Establish the Skeleton Pipelines

feature_selector = FeatureSelection()

num_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='constant', fill_value=0)),
    # ('scale_num', RobustScaler())
    ('scale_num', RobustScaler(quantile_range=(10.0, 90.0)))
    ])
cat_pipeline = Pipeline([
    # ("impute", SimpleImputer(strategy="most_frequent")),
    # ("encode_cat", OneHotEncoder(handle_unknown="ignore", sparse=False))
    ('encode_cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist'))
    ])
cols_transform = ColumnTransformer([
    ('numeric', num_pipeline, make_column_selector(dtype_include=['float64', 'int64'])),
    ('categorical', cat_pipeline, make_column_selector(dtype_include='object'))
])


In [ ]:
def get_shap_importanceNew(X_train, X_test, model, data_pipeline):

    def model_predict(X):
        return model.predict_proba(X)[:, 1] 
        return 
    
    
    # Select a small background dataset (e.g., 100 samples)
    background = X_train[np.random.choice(X_train.shape[0], 100, replace=False)]

    # Get SHAP Values
    explainer = shap.KernelExplainer(model_predict, background)
    shap_values = explainer.shap_values(X_test)

    # Get mean absolute SHAP values for global importance
    global_importance = np.abs(shap_values).mean(axis=0)
    importance_percentages = (global_importance / global_importance.sum()) * 100
    # feature_ranking = np.argsort(global_importance)[::-1]

    # Create df with names and importance
    feature_names = data_pipeline.named_steps['data_preprocess'].get_feature_names_out()
    shap_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance_%': importance_percentages
    }).sort_values(by='importance_%', ascending=False).reset_index(drop=True)
    # print(shap_importance_df.head())

    # # Get feature names for the transformed data
    # print('Top 5 Features by SHAP importance')
    # for idx in feature_ranking[:5]:
    #     print(f"{feature_names[idx]}: {global_importance[idx]:.4f}")
    
    # Plot the Top 25
    print('SHAP Values Importance')
    print(shap_importance_df.shape)
    display(shap_importance_df.head())

    importance_df = shap_importance_df.head(25)
    plt.figure(figsize=(12, 8))
    plt.barh(importance_df['feature'], importance_df['importance_%'], color='skyblue')
    plt.xlabel('Importance %')
    plt.ylabel('Feature')
    plt.title('Feature Importances (Sorted)')
    plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature at the top
    plt.show()
    
    return shap_importance_df

In [ ]:
# Evaluate A Model
#

def evaluate_model(model, X_train, X_test, y_train, y_test):

    # Need to transform the 
    xx
    

In [ ]:
# Extract the data preprocessing pipeline from the trained model
#

# Check if the model is a pipeline with preprocessing steps
if hasattr(model_randforest, 'named_steps'):
    print("Model is a pipeline with the following steps:")
    for step_name, step in model_randforest.named_steps.items():
        print(f"- {step_name}: {step}")
    
    # Extract all preprocessing steps (everything except the final classifier)
    preprocessing_steps = list(model_randforest.named_steps.items())[:-1]  # All steps except last
    
    if preprocessing_steps:
        # Create a new pipeline with just the preprocessing steps
        from sklearn.pipeline import Pipeline
        preprocessing_pipeline = Pipeline(preprocessing_steps)
        
        print(f"\nExtracted preprocessing pipeline with {len(preprocessing_steps)} steps:")
        for step_name, step in preprocessing_steps:
            print(f"- {step_name}: {step}")
        
        # Transform the data using the preprocessing pipeline
        print(f"\nOriginal X_train shape: {X_train.shape}")
        X_train_transformed = preprocessing_pipeline.transform(X_train)
        print(f"Transformed X_train shape: {X_train_transformed.shape}")
        
        print(f"Original X_test shape: {X_test.shape}")
        X_test_transformed = preprocessing_pipeline.transform(X_test)
        print(f"Transformed X_test shape: {X_test_transformed.shape}")
        
        # Display sample of transformed data
        if hasattr(X_train_transformed, 'shape') and len(X_train_transformed.shape) == 2:
            print(f"\nSample of transformed data (first 5 rows, first 10 columns):")
            if isinstance(X_train_transformed, np.ndarray):
                print(X_train_transformed[:5, :10])
            else:
                print(X_train_transformed[:5, :10])
    
    else:
        print("No preprocessing steps found in the model pipeline")
        preprocessing_pipeline = None

else:
    print("Model is not a pipeline - it's a single estimator")
    print(f"Model type: {type(model_randforest)}")
    preprocessing_pipeline = None

# xxxxxx

In [ ]:
# Establish a Model Pipeline - With Features Search
#

# Pipeline, params & grid search define
model_pipeline = Pipeline([
    ('features_select', FeatureSelection()),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1, verbose=False))
    ])

grid_params = {
    'features_select__features_detail_level': ['region', 'channel'],
    'features_select__selected_features': [['exp', 'offset'], ['cf', 'pw']],
    'classifier__criterion': ['gini', 'entropy', 'log_loss'],     # Default gini. Tree split evaluation function
    'classifier__n_estimators': [150, 175],                       # Default 100. Number of trees
    'classifier__max_depth': [2, 10, None],                       # Default none, unlimited
    'classifier__max_leaf_nodes': [50, None],                     # Default none, unlimited
    # 'randomforestclassifier__min_samples_split': [2, 5],
    # 'classifier__class_weight': [None, 'balanced']
                            # Balanced gives more importance to minority classes ... ?? Improves recall at the expense of precision
    } 

# grid_search = GridSearchCV(
#     model_pipeline, grid_params, 
#     cv=5,
#     scoring='f1'
#     )

# Stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    model_pipeline, grid_params, cv=skf, n_jobs=-1,
    scoring='precision'
    )

# Grid search run

start_time = time.perf_counter()
grid_search.fit(X_train, y_train)
duration = time.perf_counter() - start_time

# Checkpoint - Model Training Details
ml_model_pipeline_details(model_pipeline)
print('\n')
grid_search_results(grid_search, duration)
print(f'Best Model: \n{grid_search.best_estimator_}')

model_randforest_widesearch = grid_search.best_estimator_



In [ ]:
# Model Predictions & Evaluation
#
model_randforest = grid_search.best_estimator_

start_time = time.perf_counter()
y_pred = model_randforest.predict(X_test)
duration = time.perf_counter() - start_time

# Evaluate Prediction Results
print('Model Prediction Results')
print(f'Run Time: {duration:.4f}')
classification_metrics(model_randforest, X_test, y_test, y_pred)

# Feature Importance / Contribution
# feature_importance(model_randforest, features_prep_pipeline)
